In [1]:
import numpy as np

In [13]:
# Simple matrix multiplication 
def matrix_mult(A,B):
    n,m=A.shape
    C=np.zeros([n,m])
    for i in range (0,n):
        for j in range (0,m):
            for k in range(0,m):
                C[i,j]+=A[i,k]*B[k,j]
    return C        

In [14]:
# Blocks-matrix multiplication 
def muls(x,y,b):
    n=x.shape[0]
    z = np.zeros((n,n))
    r = np.zeros((b,b)) 
    u = np.zeros((b,b)) 
    v = np.zeros((b,b))
    for i in range(0, n, b):
        for j in range(0, n, b):
            for k in range(0, n, b):
                for k1 in range(0, b):
                    for k2 in range(0, b):
                        u[k1,k2] = x[i+k1,k+k2]
                        v[k1,k2] = y[k+k1,j+k2]
                for ii in range(0, b):
                    for jj in range(0, b):
                        for kk in range(0, b):
                            r[ii,jj] = r[ii,jj] + u[ii,kk]*v[kk,jj]
            for k1 in range(0, b):
                for k2 in range(0, b):
                    z[i+k1,j+k2] = r[k1,k2]
                    r[k1,k2] = 0
    return z

In [15]:
# Simple Resolution Forward Substitution
def resolve(l,b):
    n=l.shape[0]
    x=np.zeros([n,n])
    for i in range(n):
        for j in range(n):
            x[i,j]=b[i,j]
            for k in range(i):
                x[i,j]=x[i,j]-(l[i,k]*x[k,j])
            x[i,j]=x[i,j]/l[i,i]
    return x

In [11]:
#blocs-resolution with vectorisation[slicing]
def resolve_block_vect(L,B,b):
    n=L.shape[0]
    x=np.zeros([n,n])
    s=np.zeros([b,b])
    l=np.zeros([b,b])
    for i in range(0,n,b):
        l=L[i:i+b,i:i+b]
        for j in range(0,n,b):
            s=B[i:i+b,j:j+b]
            for k in range(0,i,b):
                #l=L[i:i+b,k:k+b]
                s=s-matrix_mult(L[i:i+b,k:k+b],x[k:k+b,j:j+b])
                #s=s-L[i:i+b,k:k+b]@x[k:k+b,j:j+b]
                #s=s-muls(L[i:i+b,k:k+b],x[k:k+b,j:j+b],int(b/2))
            x[i:i+b,j:j+b]=resolve(l,s)
    return x

In [27]:
#blocs-resolution without vectorisation[slicing]
def resolve_block_ohne_vect(L,B,b):
    n=L.shape[0]
    x=np.zeros([n,n])
    s=np.zeros([b,b])
    l=np.zeros([b,b])
    for i in range(0,n,b):
        l=L[i:i+b,i:i+b]
        for j in range(0,n,b):
            s=B[i:i+b,j:j+b]
            for k in range(0,i,b):
                s=s-L[i:i+b,k:k+b]@x[k:k+b,j:j+b]
            x[i:i+b,j:j+b]=resolve(l,s)
    return x

In [16]:
#resolution par blocs avec des boucles
def resolve_block_b(X,B,L,b,mb):
    N=len(B)
    r = np.zeros((b,b)) 
    l = np.zeros((b,b)) 
    s= np.zeros((b,b))
    x = np.zeros((b,b)) 
    y = np.zeros((b,b)) 
    for i in range(0, N, b):
        for k1 in range(0, b):
            for k2 in range(0, b):
                l[k1,k2] = L[i+k1,i+k2]
        for j in range(0, N, b):
            for k1 in range(0, b):
                for k2 in range(0, b):
                    s[k1,k2] = B[i+k1,j+k2]
            
            for k in range(0,i,b):
                for ii in range(0, b):
                    for jj in range(0, b):
                        x[ii,jj] = L[i+ii,k+jj]
                        y [ii,jj] = X[k+ii,j+jj]
                s-=muls(x,y,mb)
            r=resolve(l,s)
            for k1 in range(0, b):
                for k2 in range(0, b):
                    X[i+k1,j+k2] = r[k1,k2]
    return X

In [30]:
#ggenerate triangular matrix
def mx_L(n):
    l=np.random.randint(10, size=(n, n))
    l=np.tril(l)
    for i in range(n):
        l[i,i]=1
    return l

In [19]:
#main TEST
l=mx_L(100)
n=100
x=np.random.randint(10, size=(n,n))
b=l@x

In [21]:
#temps du code simple
import time
start_time = time.time()
resolve(l,b)
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.7500219345092773 seconds ---


In [22]:
#temps code blocs vecto avec multiplication par blocs
import time
start_time = time.time()
resolve_block(l,b,20)
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.7266638278961182 seconds ---


In [23]:
#temps code blocs avec boucles avec multiplication par blocs
import time
start_time = time.time()
resolve_block_b(x,b,l,20,10)
print("--- %s seconds ---" % (time.time() - start_time))


--- 0.5555500984191895 seconds ---


In [24]:
resolve(l,b)

array([[5., 1., 1., ..., 9., 6., 4.],
       [6., 2., 2., ..., 5., 2., 7.],
       [7., 0., 7., ..., 0., 7., 1.],
       ...,
       [6., 7., 2., ..., 0., 4., 6.],
       [4., 0., 1., ..., 7., 5., 3.],
       [8., 6., 1., ..., 7., 1., 0.]])

In [25]:
resolve_block(l,b,20)

array([[5., 1., 1., ..., 9., 6., 4.],
       [6., 2., 2., ..., 5., 2., 7.],
       [7., 0., 7., ..., 0., 7., 1.],
       ...,
       [6., 7., 2., ..., 0., 4., 6.],
       [4., 0., 1., ..., 7., 5., 3.],
       [8., 6., 1., ..., 7., 1., 0.]])

In [26]:
resolve_block_b(x,b,l,20,10)


array([[5, 1, 1, ..., 9, 6, 4],
       [6, 2, 2, ..., 5, 2, 7],
       [7, 0, 7, ..., 0, 7, 1],
       ...,
       [6, 7, 2, ..., 0, 4, 6],
       [4, 0, 1, ..., 7, 5, 3],
       [8, 6, 1, ..., 7, 1, 0]])

In [28]:
resolve_block_vect(l,b,20)

array([[5., 1., 1., ..., 9., 6., 4.],
       [6., 2., 2., ..., 5., 2., 7.],
       [7., 0., 7., ..., 0., 7., 1.],
       ...,
       [6., 7., 2., ..., 0., 4., 6.],
       [4., 0., 1., ..., 7., 5., 3.],
       [8., 6., 1., ..., 7., 1., 0.]])

In [29]:
resolve_block_ohne_vect(l,b,20)

array([[5., 1., 1., ..., 9., 6., 4.],
       [6., 2., 2., ..., 5., 2., 7.],
       [7., 0., 7., ..., 0., 7., 1.],
       ...,
       [6., 7., 2., ..., 0., 4., 6.],
       [4., 0., 1., ..., 7., 5., 3.],
       [8., 6., 1., ..., 7., 1., 0.]])